In [1]:
import cv2 # opencv 사용
import numpy as np

def grayscale(img): # 흑백이미지로 변환
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def canny(img, low_threshold, high_threshold): # Canny 알고리즘
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size): # 가우시안 필터
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices, color3=(255,255,255), color1=255): # ROI 셋팅

    mask = np.zeros_like(img) # mask = img와 같은 크기의 빈 이미지
    
    if len(img.shape) > 2: # Color 이미지(3채널)라면 :
        color = color3
    else: # 흑백 이미지(1채널)라면 :
        color = color1
        
    # vertices에 정한 점들로 이뤄진 다각형부분(ROI 설정부분)을 color로 채움 
    cv2.fillPoly(mask, vertices, color)
    
    # 이미지와 color로 채워진 ROI를 합침
    ROI_image = cv2.bitwise_and(img, mask)
    return ROI_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=2): # 선 그리기(파란색)
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def draw_fit_line(img, lines, color=[255, 0, 0], thickness=10): # 대표선 그리기
        cv2.line(img, (lines[0], lines[1]), (lines[2], lines[3]), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap): # 허프 변환
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    #line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    #draw_lines(line_img, lines)

    return lines
    #return line_img

def weighted_img(img, initial_img, α=1, β=1., λ=0.): # 두 이미지 operlap 하기
    return cv2.addWeighted(initial_img, α, img, β, λ)

def get_fitline(img, f_lines): # 대표선 구하기   
    lines = np.squeeze(f_lines)
    lines = lines.reshape(lines.shape[0]*2,2)
    rows,cols = img.shape[:2]
    output = cv2.fitLine(lines,cv2.DIST_L2,0, 0.01, 0.01)
    vx, vy, x, y = output[0], output[1], output[2], output[3]
    x1, y1 = int(((img.shape[0]-1)-y)/vy*vx + x) , img.shape[0]-1
    x2, y2 = int(((img.shape[0]/2+100)-y)/vy*vx + x) , int(img.shape[0]/2+100)
    
    result = [x1,y1,x2,y2]
    return result


def expression(x1,y1,x2,y2,x3,y3,x4,y4):
    m_a = (y2 - y1) / (x2 -x1)
    m_b = (y4 - y3) / (x4 - x3)
    n_a = -((y2 - y1) / (x2 - x1) * x1 ) + y1
    n_b = -((y4 - y3) / (x4 -x3) * x3 ) + y3
    x = (n_b - n_a) / (m_a - m_b) 
    y = m_a * ((n_b - n_a) / (m_a - m_b)) + n_a 
    return x,y

cap = cv2.VideoCapture('solidWhiteRight.mp4') # 이미지 읽기
#cap = cv2.VideoCapture('testvideo.mp4') # 이미지 읽기 -> 소실점 too many indices 에러 

while(cap.isOpened()):

    ret,image = cap.read()
    #image = cv2.resize(image, (640, 360)) # window 사이즈 조정

    height, width = image.shape[:2] # 이미지 높이, 너비

    m_width = int(width/2)

    gray_img = grayscale(image) # 흑백이미지로 변환
    
    blur_img = gaussian_blur(gray_img, 3) # Blur 효과
   
    canny_img = canny(blur_img, 70, 210) # Canny edge 알고리즘

    vertices = np.array([[(50,height),(width/2-45, height/2+60), (width/2+45, height/2+60), (width-50,height)]], dtype=np.int32)
    ROI_img = region_of_interest(canny_img, vertices) # ROI 설정
    #cv2.imshow('result',ROI_img)

    rho = 1
    theta = 1 * np.pi/180
    threshold = 30
    line_arr = hough_lines(ROI_img, rho, theta, threshold, 10, 20) # 허프 변환
    #print(line_arr.shape) # (8,1,4), (7,1,4) ...(3,1,4) ...
    #print(line_arr[:]) # [ [[523 330 870 538]], [[707 450 848 538]], ...]
    
    line_arr = np.squeeze(line_arr) # remove single dimension
    #print(line_arr.shape) # (8, 4), (7, 4), ...(3, 4) ...
    #print(line_arr[:]) # [[523 330 870 538], [707 450 848 538] ...]
    
    
# 기울기 구하기 (arctan(y,x)이용)
    # arr[: , 3]  => 모든행에서, 열의 인덱스가 3인값(=4열)을 추출 
    slope_degree = np.arctan2(line_arr[:,1] - line_arr[:,3], line_arr[:,0] - line_arr[:,2]) * 180 / np.pi
    
# 수평 기울기 제한
    line_arr = line_arr[np.abs(slope_degree)<165]
    slope_degree = slope_degree[np.abs(slope_degree)<165]
# 수직 기울기 제한
    line_arr = line_arr[np.abs(slope_degree)>95]
    slope_degree = slope_degree[np.abs(slope_degree)>95]
# 필터링된 직선 버리기
    L_lines, R_lines = line_arr[(slope_degree>0),:], line_arr[(slope_degree<0),:]
    temp = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
    L_lines, R_lines = L_lines[:,None], R_lines[:,None]
    #print(temp[:])
    '''
    temp
    [[[0 0 0]
      [0 0 0]
      [0 0 0]
      ...
      [0 0 0]
      [0 0 0]
      [0 0 0]]

     [[0 0 0]
      [0 0 0]
      [0 0 0]
      ...
      [0 0 0]
      [0 0 0]
      [0 0 0]]
    '''
    
    
# 왼쪽, 오른쪽 각각 대표선 구하기
    left_fit_line = get_fitline(image,L_lines)
    right_fit_line = get_fitline(image,R_lines)
    #print(left_fit_line) # [158, 539, 388, 370]...
    
# 대표선 그리기
    draw_fit_line(temp, left_fit_line)
    draw_fit_line(temp, right_fit_line)

    #print(left_fit_line[0])
    #print(left_fit_line[1])
    #print(left_fit_line[2])
    #print(left_fit_line[3])

    #print(right_fit_line[0])
    #print(right_fit_line[1])
    #print(right_fit_line[2])
    #print(right_fit_line[3])

    vanishing_point = expression(left_fit_line[0],left_fit_line[1],left_fit_line[2],left_fit_line[3],right_fit_line[0],right_fit_line[1],right_fit_line[2],right_fit_line[3])
    #print(vanishing_point) # (476.9880952380953, 304.61309523809524)...

    
    v_x = int(vanishing_point[0])
    v_y = int(vanishing_point[1])
    #print(v_x, v_y) # 476 304 ...
    
    result = weighted_img(temp, image) # 원본 이미지(=image)에 검출된 선(=temp) overlap
    cv2.circle(result, (v_x,v_y), 6, (0,0,255), -1) # cv2.circle(image, center_coordinates, radius, color, thickness)
    
    #circle 기준선 
    cv2.line(result,(m_width,0),(m_width,300),(255,255,0),5) # cv2.line(image, start_point, end_point, color, thickness)

    
    if(v_x > m_width): # 소실점의 x좌표가 중앙선보다 오른쪽에 있을때
        print("Right!!!")
        #cv2.circle(result,(1000,50), 6,(0,0,255),-1) # (1000,50)에 circle 찍어라 

    if(v_x < m_width): # 소실점의 x좌표가 중앙선보다 왼쪽에 있을때
        print("Left!!!")
        #cv2.circle(result,(100,50), 6,(0,0,255),-1) # (100,50)에 circle 찍어라 
    
    
    cv2.imshow('result',result) # 결과 이미지 출력
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Right!!!
Left!!!
Right!!!
Left!!!
Right!!!
Left!!!
Right!!!
Left!!!
Right!!!
Right!!!
Right!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Right!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Left!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Ri